In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
import os

# === Parameters ===
static_graph = nx.read_adjlist("../BA-200.txt", nodetype=int)
N = static_graph.number_of_nodes()
m = 20
timesteps = 300
beta = 0.08
mu = 0.1
num_runs = 100
activity = 0.01

# === Run Simulation Function ===
def run_sir_once():
    # static_graph = nx.read_adjlist("BA-200.txt.txt", nodetype=int)
    activity_rates = {node: activity for node in static_graph.nodes}
    states = {i: 'I' if random.random() < 0.05 else 'S' for i in range(N)}

    counts = {'S': [], 'I': [], 'R': []}
    extinction_time = timesteps  # default if infection never dies

    for step in range(timesteps):
        # Generate temporal layer
        G_temporal = nx.Graph()
        for i in range(N):
            G_temporal.add_node(i)
            if random.random() < activity_rates[i]:
                targets = random.sample([j for j in range(N) if j != i and not static_graph.has_edge(i, j)], m)
                for j in targets:
                    G_temporal.add_edge(i, j)

        # Combine graphs
        G_combined = nx.Graph()
        G_combined.add_edges_from(static_graph.edges())
        G_combined.add_edges_from(G_temporal.edges())

        # Infection update
        new_states = states.copy()
        for node in range(N):
            if states[node] == 'I':
                if random.random() < mu:
                    new_states[node] = 'R'
            elif states[node] == 'S':
                for neighbor in G_combined.neighbors(node):
                    if states[neighbor] == 'I' and random.random() < beta:
                        new_states[node] = 'I'
                        break
        states = new_states

        # Count states
        current_I = sum(1 for s in states.values() if s == 'I')
        counts['S'].append(sum(1 for s in states.values() if s == 'S'))
        counts['I'].append(current_I)
        counts['R'].append(sum(1 for s in states.values() if s == 'R'))

        if current_I == 0 and extinction_time == timesteps:
            extinction_time = step

    return counts, extinction_time


# === Run Multiple Simulations ===
print(f"▶ Running {num_runs} SIR simulations for {timesteps} steps...")
all_S, all_I, all_R = [], [], []
extinction_times = []
peak_times = []
peak_values = []

for _ in range(num_runs):
    counts, die_out = run_sir_once()
    all_S.append(counts['S'])
    all_I.append(counts['I'])
    all_R.append(counts['R'])
    extinction_times.append(die_out)

    peak_idx = np.argmax(counts['I'])
    peak_val = max(counts['I'])
    peak_times.append(peak_idx)
    peak_values.append(peak_val)

▶ Running 100 SIR simulations for 300 steps...
Saved SIR plot as sir_average_curve_m=20.png

m=20
Ai=0.01
extinction=78.15 ± 22.42
peak infection: 17.29 ± 11.71
peak infection size: 28.79 ± 9.42
Saved summary to sir_summary_results.csv


In [16]:
# === Convert to numpy arrays ===
all_S = np.array(all_S)
all_I = np.array(all_I)
all_R = np.array(all_R)

# === Compute Averages and StdDev ===
mean_S = np.mean(all_S, axis=0)
mean_I = np.mean(all_I, axis=0)
mean_R = np.mean(all_R, axis=0)

std_S = np.std(all_S, axis=0)
std_I = np.std(all_I, axis=0)
std_R = np.std(all_R, axis=0)

# === Infection Extinction Stats ===
avg_extinction_time = np.mean(extinction_times)
std_extinction_time = np.std(extinction_times)

avg_peak_time = np.mean(peak_times)
std_peak_time = np.std(peak_times)
avg_peak_val = np.mean(peak_values)
std_peak_val = np.std(peak_values)

# === Plot ===
t = np.arange(timesteps)
plt.figure(figsize=(10, 6))

plt.plot(t, mean_S, label='Susceptible', color='skyblue', linewidth=2)
plt.fill_between(t, mean_S - std_S, mean_S + std_S, color='skyblue', alpha=0.3)

plt.plot(t, mean_I, label='Infected', color='red', linewidth=2)
plt.fill_between(t, mean_I - std_I, mean_I + std_I, color='red', alpha=0.3)

plt.plot(t, mean_R, label='Recovered', color='gray', linewidth=2)
plt.fill_between(t, mean_R - std_R, mean_R + std_R, color='gray', alpha=0.3)

plt.title(f"Average SIR (Average extinction time: {avg_extinction_time:.2f} ± {std_extinction_time:.2f} steps, peak size: {avg_peak_val:.2f} ± {std_peak_val:.2f},  m={m})")
plt.xlabel("Time Step")
plt.ylabel("Number of Nodes")
plt.ylim(0, N)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f"sir_average_curve_m={m}.png", dpi=300)
plt.close()
print(f"Saved SIR plot as sir_average_curve_m={m}.png")

# === Save to DataFrame
summary_data = {
    'm': [m],
    'Ai': [activity],
    'extinction_time': [avg_extinction_time],
    'extinction_std': [std_extinction_time],
    'peak_time': [avg_peak_time],
    'peak_time_std': [std_peak_time],
    'peak_size': [avg_peak_val],
    'peak_size_std': [std_peak_val],
    'total recovered': [mean_R.max()]
}
df = pd.DataFrame(summary_data)

df

Saved SIR plot as sir_average_curve_m=20.png


,m,Ai,extinction_time,extinction_std,peak_time,peak_time_std,peak_size,peak_size_std,total recovered
0,20,0.01,78.15,22.423815,17.29,11.710931,28.79,9.421566,88.53


In [ ]:
# === Append to CSV
csv_file = "sir_summary_results.csv"
if os.path.exists(csv_file):
    existing_df = pd.read_csv(csv_file)
    combined_df = pd.concat([existing_df, df], ignore_index=True)
else:
    combined_df = df

combined_df.to_csv(csv_file, index=False)
print(f"Saved summary to {csv_file}")